Count the number of lipid droplets per cell in iPSC neurons to determine how various genetic and treatment conditions affect lipid morphology.

Import Libraries

In [1]:
import czifile
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display
from skimage.color import label2rgb, rgb2hsv
from skimage.filters import gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk, local_maxima
from scipy.ndimage import binary_fill_holes
from skimage.feature import blob_log
from skimage.color import rgb2gray
from skimage.io import imshow
from skimage.draw import circle_perimeter
from matplotlib.colors import LogNorm
import scipy.ndimage as ndi
import pandas as pd
import os
from IPython.display import clear_output
from cellpose import models, io, plot
model = models.Cellpose(model_type='cyto3')
import time

Define Sub Functions

In [2]:
def process_file_noseg(file_path, basename):
    """
    Processes a .czi file, segments and identifies nuclei and lipid droplets without segmentation,
    and visualizes the results.

    Args:
        file_path (str): The path to the .czi file.
        basename (str): The base name of the file.

    Returns:
        tuple: Number of nuclei and lipid droplets identified in the image.
    """
    image = czifile.imread(file_path)
    image_squeezed = np.squeeze(image)
    first_frame = image_squeezed[0, :, :]
    second_frame = image_squeezed[1, :, :]

    red_channel = gaussian(first_frame, sigma=2)
    blue_channel = gaussian(second_frame, sigma=2) > 0.2

    labeled_image = label(blue_channel)
    nuclei = [region.area for region in regionprops(labeled_image)]
    nuclei = [x for x in nuclei if x > 500]

    threshold = np.percentile(red_channel, 95) + 0.1
    inclusions = red_channel > threshold

    labeled_image = label(inclusions)
    lipids_list = [region.area for region in regionprops(labeled_image)]

    return len(nuclei), len(lipids_list)



Main Function

In [3]:
def main():
    """
    Main function to process multiple .czi files in a folder, identify nuclei and lipid droplets,
    and save the results to an Excel file.
    """
    folder_path = r"test_images"
    basenames = []
    nuclei = []
    inclusions = []

    for well_image in os.listdir(folder_path):
        if well_image.lower().endswith(".czi"):  # Filter for CZI files
            well_image_path = os.path.join(folder_path, well_image)
            well_image_base_name = os.path.basename(well_image)[:-4]
            num_nuclei, num_inclusions = process_file_noseg(well_image_path, well_image_base_name)
            if num_nuclei == 0: #if there are no nuclei, skip the image
                continue
            nuclei.append(num_nuclei)
            inclusions.append(num_inclusions)
            basenames.append(well_image_base_name)

    # Save results to Excel
    df = pd.DataFrame({
        "Filename": basenames,
        "Nuclei": nuclei,
        "Lipid Droplets": inclusions
    })

    combined_csv_path = os.path.join("NEURONS.xlsx")
    df.to_excel(combined_csv_path, index=False)

if __name__ == "__main__":
    main()